# **Telecom Customer Churn Prediction Project**

------------------------
# Tabel of contents

1- [Introduction](#1--Introduction)  
2- [Loading Libraries and Data](#2--Loading-Libraries-and-Data)  
3- [Data Understanding](#3--Data-Understanding)  
4- [Data Cleaning](#4--Data-Cleaning)  
5- [Exploratory Data Analysis](#5--Exploratory-Data-Analysis) 
* [Bussiness Insightes](#Bussiness-Insightes:)
* [Data Preparation](#Data-Preparation:)
* [EDA for Machine Learning](#EDA-for-Machine-Learning:)
* [Key Insights](#Key-Insights)
* [Recommenditions](#Recommenditions)

6- [Data Preprocessing and Feature Engineering](#6--Data-Preprocessing-and-Feature-Engineering)   
7- [Machine Learning Modeling](#7--Machine-Learning-Modeling)  
*  [KNN](#KNN)
*  [Random Forest](#Random-Forest)

8- [Summary](#8--Summary)  


--------------------------
# 1- Introduction
<a id="Introduction"></a>


### Objective:

We will briefly describe the project's goal of predicting customer churn in the telecom industry.

### Business Problem:

In this Project, we will try to explain why churn prediction is important and how it can help the business retain customers.

--------------------------
# 2- Loading Libraries and Data
<a id="Loading-Libraries-and-Data"></a>

### Importing Libraries:

In [7]:
# Data manipulation and analysis
import pandas as pd
import numpy as np

# Data visualization
import matplotlib.pyplot as plt
import seaborn as sns
sns.set_theme(style='white') 
%matplotlib inline
import plotly.express as px 
import plotly.graph_objects as go
from plotly.subplots import make_subplots
import plotly.io as pio
pio.renderers.default='notebook_connected'

# Preprocessing
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import StandardScaler, LabelEncoder

# Model building and evaluation
from sklearn.ensemble import RandomForestClassifier
from sklearn.linear_model import LogisticRegression
from sklearn.tree import DecisionTreeClassifier
from sklearn.neighbors import KNeighborsClassifier 
from sklearn.pipeline import Pipeline
from sklearn.metrics import accuracy_score, confusion_matrix, classification_report, precision_score, recall_score, f1_score
from sklearn.model_selection import StratifiedShuffleSplit
from sklearn.utils import resample

# Hyperparameter tuning
from sklearn.model_selection import GridSearchCV

# Import the required function
from scipy.stats import chi2_contingency,wilcoxon, mannwhitneyu, skew
from scipy.stats import chi2_contingency 
from itertools import combinations
import scipy.stats as ss

# Display plots inline
%matplotlib inline

# Configure seaborn
sns.set(style="whitegrid")

import warnings
warnings.filterwarnings('ignore') 

AttributeError: `np.unicode_` was removed in the NumPy 2.0 release. Use `np.str_` instead.

### Loading Data:

In [ ]:
# Load the CSV file
url = 'https://raw.githubusercontent.com/ahosnyyy/capstone_project/refs/heads/main/data/telco-customer-churn.csv'
df = pd.read_csv(url)

----------------------
# 3- Data Understanding
<a id="Data-Understanding"></a>

- Each row represents a customer, each column contains customer’s attributes described on the column Metadata.

In [ ]:
# removing the max columns limit to better read the data
pd.set_option('display.max_columns', None)
df.head()      # Display the first few rows

- Next, we will check the structure of the dataset, including data types and missing values.

In [ ]:
# Check data structure
df.shape

### Data Overview:

 The dataset contains information about **7,043** telecom customers and their service usage patterns, demographics, and whether they churned or not. It has **21** columns, covering aspects like customer demographics, account details, services they have subscribed to, and their churn status. **Below are some key features:**

*  **Customer Information:**

   * `customerID`: Unique identifier for each customer.
   * `gender`: Gender of the customer (Male/Female).
   * `SeniorCitizen`: Whether the customer is a senior citizen (1 for Yes, 0 for No). 
   * `Partner` and `Dependents`: Whether the customer has a partner or dependents.   

* **Service Information:**
  * `PhoneService`, `MultipleLines`: Whether the customer has phone service and multiple lines.
  * `InternetService`, `OnlineSecurity`, `OnlineBackup`, `DeviceProtection`, `TechSupport`, `StreamingTV`, `StreamingMovies`: Various internet services and support options the customer uses.

* **Account Information:**

  * `Contract`: Type of contract the customer has (Month-to-month, One year, Two years).
  * `PaperlessBilling`: Whether the customer has opted for paperless billing.
  * `PaymentMethod`: The customer’s method of payment (e.g., Electronic check, Mailed check).
  * `MonthlyCharges` and `TotalCharges`: Monthly and total charges for the customer.
  * `tenure`: Indicates the total amount of months that the customer has been with the company.

* **Target Variable:**

  * `Churn`: Whether the customer churned (Yes/No).
  
This dataset is well-suited for churn prediction, providing various service and account attributes that can be used to understand patterns in customer churn.

In [ ]:
df.info()

In [ ]:
df.columns.values

In [ ]:
df.dtypes

In [ ]:
df.describe()    # Summary statistics for numerical features

In [ ]:
# Check for Null values in the dataset
df.isnull().sum()

**Checking duplicates**

In [ ]:
duplicates = df.duplicated()
print(duplicates.sum())  # This will show the total number of duplicate rows

--------------------------
# 4- Data Cleaning
<a id="Data-Cleaning"></a>

From the initial data overview, we can find some indirect missingness in our data (which can be in form of blank spaces), particularly `TotalCharges`.  
The column `TotalCharges` might be of type `object` instead of `float` due to the presence of missing or bad data.  
Let's see that !

In [ ]:
df['TotalCharges'] = pd.to_numeric(df['TotalCharges'], errors='coerce') # coerce => If the data null will put NAN instead of error

In [ ]:
df.isnull().sum()

- Here we see that the TotalCharges has **11** missing values. Let's check this data.

In [ ]:
df[np.isnan(df['TotalCharges'])]

- It can also be noted that the `Tenure` column is `0` for these entries even though the `MonthlyCharges` column is not empty.  
Let's see if there are any other `0` values in the `tenure` column.

In [ ]:
df[df['tenure'] == 0].index

- There are no additional missing values in the `Tenure` column.  
- Let's delete the rows with missing values in `TotalCharges` columns since there are only **11** rows and deleting them will not affect the data.

In [ ]:
# Drop rows where TotalCharges has missing values (NaN)
df.dropna(subset=['TotalCharges'], inplace=True)

# Verify if any missing values remain
df.isnull().sum()

------------------------
# 5- Exploratory Data Analysis
<a id="Exploratory-Data-Analysis"></a>

## Bussiness Insightes:
<a id="Bussiness-Insightes:"></a>

### Overview of the Dataset

- Description for summary statistics for numerical features after updating data types.

In [ ]:
df[['tenure','MonthlyCharges','TotalCharges']].describe()

### **A) Distribution of Target Variable (`Churn`)**
  
We will analyze the distribution of the target variable, `Churn`, to understand if the dataset is balanced or imbalanced.

In [ ]:
main_color = ['#26577C','#E55604','#CDC2AE','orange','#867070']
df_t = df['Churn'].value_counts().reset_index().set_axis(['Churn','Count'],axis=1)
df_t['Churn'] = df_t['Churn'].replace({0:'Not Churn',1:'Churn'})
z = [main_color[0],main_color[1]]
fig = go.Figure(data=[go.Pie(labels=df_t['Churn'], values=df_t['Count'], hole=.3)]
            ).update_layout(height=550,title={'text': 'Customer Churn','y':0.96,'x':0.485,'xanchor': 'center','yanchor': 'top'},font=dict( size=15), template='plotly_white'
                            ).update_traces(marker=dict(colors=z))
fig.show()

**Notes:**   
- The dataset is somewhat imbalanced, with a significantly higher number of customers who did not churn (73.4 %).
- This imbalance is common in customer churn datasets but We will have to take that into account while splitting the dataset.


--------------------------
**Functions**

In [ ]:
## I will put some functions here so as not to interfere

# function to calculate categorical data association
def get_corr_mat(df, f=chi2_contingency):
    columns = df.columns
    dm = pd.DataFrame(index=columns, columns=columns)
    for var1, var2 in combinations(columns, 2):
        cont_table = pd.crosstab(df[var1], df[var2], margins=False)
        chi2_stat = cramers_v(cont_table.values)
        dm.loc[var2, var1] = chi2_stat
        dm.loc[var1, var2] = chi2_stat
    dm.fillna(1, inplace=True)
    return dm

def cramers_v(confusion_matrix):
    """ Calculate Cramer's V statistic for categorical-categorical association. """
    chi2 = ss.chi2_contingency(confusion_matrix)[0]
    n = confusion_matrix.sum()
    phi2 = chi2 / n
    r, k = confusion_matrix.shape
    phi2corr = max(0, phi2 - ((k-1)*(r-1))/(n-1))
    rcorr = r - ((r-1)**2)/(n-1)
    kcorr = k - ((k-1)**2)/(n-1)
    return np.sqrt(phi2corr / min((kcorr-1), (rcorr-1)))

#################################################################################

# Categorical data visualization function
def cat_viz(list_of_var):
    bucket1 = [] 
    bucket2 = []
    bucket3 = []
    bucket4 = []
    
    for i in list_of_var:
        bucket1 = bucket1 + [i] * len(df[i].unique())
        for y in df[i].unique():
            bucket2.append(y)
            bucket3.append(df[(df['Churn'] == 'Yes') & (df[i] == y)][i].count())
            bucket4.append(df[(df['Churn'] == 'No') & (df[i] == y)][i].count())
    
    bucket5 = []
    for i, j in zip(bucket4, bucket3):
        bucket5.append([round((i / (i + j)) * 100, 2), round((j / (i + j)) * 100, 2)])
    
    fig = go.Figure()

    # Not Churn
    fig.add_trace(go.Bar(
        x=[bucket1, bucket2], 
        y=bucket4, 
        marker_color=main_color[0], 
        name="Not Churn", 
        text=[f"{j}<br>%{i[0]}" for i, j in zip(bucket5, bucket4)]
    ))

    # Churn
    fig.add_trace(go.Bar(
        x=[bucket1, bucket2], 
        y=bucket3, 
        marker_color=main_color[1], 
        name="Churn", 
        text=[f"{j}<br>%{i[1]}" for i, j in zip(bucket5, bucket3)]
    ))

    fig.update_layout(title_text="Categorical Data vs Churn", 
                      title_x=0.5, template='plotly_white')

    # Correlation matrix for the categorical variables
    cat_corr = get_corr_mat(df[list_of_var + ['Churn']])
    
    # Plot the heatmap
    plt.figure(figsize=(5, 5))
    sns.heatmap(cat_corr, annot=True, fmt='.2f', square=True, cmap='Spectral')
    plt.show()

    return fig

#########################################################################################

# Numerical data visualization function
def num_viz(var):
    skew_ = round(skew(df[var], bias=False), 3)
    
    fig = px.histogram(
        df, 
        x=var,
        marginal="box", 
        hover_data=df.columns, 
        color='Churn',
        color_discrete_map={'No': main_color[0], 'Yes': main_color[1]}
    ).update_layout(
        autosize=True,
        height=360, 
        width=900,
        margin=dict(l=0, r=20, b=0, t=30, pad=10),
        title_text=f"<b>{var} | Skewness: {skew_}",
        title_x=0.5, 
        template='plotly_white'
    )
    return fig

-----------------------------
### **B) Demographics**

- This analysis refers to customers. We will analyze **customer information** to discover the correlations and relation to `churn`.

#### Demographic features are all categorical:  

`gender`, `SeniorCitizen`, `Partner`, `Dependents`

In [ ]:
cat_viz(['gender', 'SeniorCitizen', 'Partner', 'Dependents']).show()

**Notes**  

- The proportion of Gender between Male and Female towards Churn looks identical, Gender has no association with customers who will churn or not.
- It can be observed that the fraction of senior citizen is very less.
- Senior Citizens are much more likely to churn.
- Customers without partner are more likely to churn.
- Customers who do not have dependents are more likely to churn.


### **C) Service** 

- This analysis refers to `Service`. We will analyze service information to discover the correlations and relation to `churn`.

#### All Features are categorical:

`PhoneService`,`MultipleLines`, `InternetService`, `OnlineSecurity`, `OnlineBackup`, `DeviceProtection`, `TechSupport`, `StreamingTV`, `StreamingMovies`

In [ ]:
# List of service-related columns
serv = ['PhoneService', 'MultipleLines', 'InternetService', 'OnlineSecurity', 
        'OnlineBackup', 'DeviceProtection', 'TechSupport', 'StreamingTV', 'StreamingMovies']

map_serv = {'Yes':1,'No':0,'No phone service':0,'DSL':1,'Fiber optic':1,'No internet service':0}


# Data frame for visualization of Count of Services usage
df_serv = pd.DataFrame(pd.DataFrame([df[i].map(map_serv) for i in serv]).T.sum(axis=0)).reset_index().set_axis(
    ['Services','Count'], axis=1).sort_values(by='Count', ascending=False, ignore_index=True)

# Highlight the top 3 services
df_serv['Color'] = main_color[0]
df_serv['Color'][:3] = main_color[1]  # highlight top 3 services

# Visualization of Count of Services usage
fig = go.Figure([go.Bar(y=df_serv['Services'], x=df_serv['Count'],
                        text=[f'{i} ({j}%)' for i, j in zip(df_serv['Count'], 
                        [round((i / df_serv['Count'].sum()) * 100, 2) for i in df_serv['Count']])],
                        textposition='outside',
                        marker={'color': df_serv['Color']},
                        orientation='h')])

# Update layout
fig.update_layout(autosize=True,
                  height=500,
                  margin=dict(l=10, r=10, b=10, t=40, pad=0),
                  xaxis_range=[0, df_serv['Count'].max() + (df_serv['Count'].max() / 2)],
                  template='plotly_white',
                  title=dict(text='Count of Services Usage', font=dict(size=20), x=0.5),
                  yaxis={'categoryorder': 'array', 'categoryarray': df_serv.sort_values(by='Count', ascending=True)['Services'].to_list()})

# Show the figure
fig.show()


**Notes**  

- `PhoneService`, `InternetService`, and `MultipleLines` are the 3 most popular services, next we will try to find out how these three variables relate to `churn`

In [ ]:
cat_viz(['PhoneService','InternetService','MultipleLines']).show()

**Notes**  
- Customers who use fiber optic internet services have a high chance of Churn.

In [ ]:
#cat_viz(['StreamingMovies','StreamingTV', 'OnlineBackup']).show()

In [ ]:
#cat_viz(['DeviceProtection', 'TechSupport', 'OnlineSecurity']).show()

### D) Account 
- These analysis refers to `customer Account` and the process he follow to use the service. We will analyze Account information to discover the correlations and relation to `churn`.



#### For categorical features 

`Contract`,  `PaymentMethod`, `PaperlessBilling`

In [ ]:
custom_colors = ['#26577C','#E55604','#CDC2AE'] 

fig = px.histogram(df, 
                   x="Churn", 
                   color="Contract", 
                   barmode="group", 
                   title="<b>Customer Contract Distribution<b>",
                   color_discrete_sequence=custom_colors)

# Update layout
fig.update_layout(width=700, height=500, bargap=0.1)

# Show the figure
fig.show()


**Notes** 
- About `75%` of customer with Month-to-Month Contract opted to move out as compared to `13%` of customrs with One Year Contract and `3%` with Two Year Contract.

In [ ]:
labels = df['PaymentMethod'].unique()
values = df['PaymentMethod'].value_counts()

custom_colors_2 = ['#26577C','#E55604','#CDC2AE','orange'] 

fig = go.Figure(data=[go.Pie(labels=labels, values=values, hole=.3, 
                             marker=dict(colors=custom_colors_2))])
fig.update_layout(title_text="<b>Payment Method Distribution</b>")
fig.show()

In [ ]:
custom_colors_3 = {
    'Electronic check': '#26577C',
    'Mailed check': '#E55604',
    'Bank transfer (automatic)': '#CDC2AE',
    'Credit card (automatic)': 'orange'
}

fig = px.histogram(df, 
                   x="Churn", 
                   color="PaymentMethod", 
                   title="<b>Customer Payment Method distribution w.r.t. Churn</b>", 
                   color_discrete_map=custom_colors_3)

# Update layout
fig.update_layout(width=700, height=500, bargap=0.1)

# Show the figure
fig.show()


**Notes:**  
- Major customers who moved out were having Electronic Check as Payment Method.
- Customers who opted for Credit-Card automatic transfer or Bank Automatic Transfer and Mailed Check as Payment Method were less likely to move out.

In [ ]:
color_map = {"Yes": '#26577C', "No": '#E55604'}
fig = px.histogram(df, x="Churn", color="PaperlessBilling",  title="<b>Chrun distribution w.r.t. Paperless Billing</b>", color_discrete_map=color_map)
fig.update_layout(width=700, height=500, bargap=0.1)
fig.show()

**Notes:**  
- Customers with Paperless Billing are most likely to churn.


#### For Numerical features:

In [ ]:
num_viz('tenure').show()

**Notes:**
- New customers are more likely to churn.

In [ ]:
num_viz('MonthlyCharges').show()

**Notes:**  
- Customers with higher Monthly Charges are also more likely to churn.

In [ ]:
num_viz('TotalCharges').show()

**Notes:**
- If we look at the marginal boxplot visualization above, it can be seen that `tenure` and `MonthlyCharges` do not have different data distribution shapes,  
  This is one indication that this feature will be a good predictor.

## Data Preparation:
<a id="Data-Preparation:"></a>

### A) Checking the Unique Values for Object Features:

In [ ]:
# Select only object columns
object_columns = df.select_dtypes(include=['object'])

# Get unique values and their counts for each object column
unique_values_info = {col: {
    "unique_values": object_columns[col].unique(),
    "num_unique": object_columns[col].nunique()
} for col in object_columns.columns}

# Print the unique values and their counts
for column, info in unique_values_info.items():
    print(f"Unique values in '{column}':")
    print(info["unique_values"])
    print(f"Number of unique values: {info['num_unique']}")
    print()  # Print a new line for better readability

### B) Standardizing Categorical Features for Consistency:

- In the `OnlineSecurity` and Similar Features, we observed three unique values:
  - **Yes**: Indicates the customer has OnlineSecurity.
  - **No**: Indicates the customer does not have OnlineSecurity.
  - **No internet service**: Indicates the customer does not have any Internet service.

- To enhance data consistency:
  - We will replace the **No internet service** value with **No**, as it aligns with the binary nature of the feature.
  - This transformation will improve the clarity and usability of the data for analysis and machine learning models.

- Additionally, since we have a separate column **InternetService** that indicates whether the customer has internet service, standardizing these values helps maintain uniformity across related features.

In [ ]:
df.loc[df['InternetService'] == 'No', 
                     ['InternetService', 'OnlineSecurity', 'OnlineBackup', 
                      'DeviceProtection', 'TechSupport', 'StreamingTV', 
                      'StreamingMovies']]

- Similar to the `MultipleLines` feature case we are appling the same solution to the similar features

In [ ]:
# List of columns to replace 'No internet service' and 'No phone service:' with 'No'
columns_to_replace = [
    'MultipleLines',
    'OnlineSecurity',
    'OnlineBackup',
    'DeviceProtection',
    'TechSupport',
    'StreamingTV',
    'StreamingMovies'
]

# Replace 'No internet service' with 'No' in specified columns
for column in columns_to_replace:
    df[column] = df[column].replace({'No internet service': 'No', 'No phone service': 'No'})

### C) Convert Binary Categorical Variables:

In [ ]:
# Convert 'Yes'/'No' columns to 0 and 1
binary_columns = [
    'Partner',
    'Dependents',
    'PhoneService',
    'MultipleLines',
    'OnlineSecurity',
    'OnlineBackup',
    'DeviceProtection',
    'TechSupport',
    'StreamingTV',
    'StreamingMovies',
    'PaperlessBilling',
    'Churn'
]

# Mapping Yes/No to 1/0
for col in binary_columns:
    df[col] = df[col].map({'Yes': 1, 'No': 0})

# Convert 'gender' column separately
df['gender'] = df['gender'].map({'Male': 1, 'Female': 0})

# Check data types
print(df.dtypes)

-------------------------
## EDA for Machine Learning:
<a id="EDA-for-Machine-Learning:"></a>

In [ ]:
# Deviding the data into two groups by creating two masks to easly select them 
churn = df['Churn'] == 1
no_churn = df['Churn'] == 0

In [ ]:
# Displaying the mean for numeric features, to cross-examine them according to churn status.
means = [df[churn].mean(numeric_only=True),df[no_churn].mean(numeric_only=True)]
pd.DataFrame(means, index=['churn', 'no_churn'])

--------------------------
**Functions**

In [ ]:
# making a function to plot a histogram for each group against 1 feature
def hist_1d(df, col_name, title= None, xlabel= None, ylabel=None):
    
    df[col_name][no_churn].hist(alpha=0.7, label= 'No churn', color= '#26577C');
    df[col_name][churn].hist(alpha=0.7, label= 'churn', color= '#E55604');
    
    # title and labels
    plt.title(title, fontsize = 20)
    plt.xlabel(xlabel,fontsize = 15)
    plt.ylabel(ylabel, fontsize = 15)
    
    # legend
    plt.legend();

In [ ]:
# making a function to plot a bar chart for each group against 2 variables
def bar_2d(df, x_col_name, y_col_name, title= None, xlabel= None, ylabel=None):
    
    df[no_churn].groupby(x_col_name)[y_col_name].mean().plot(kind='bar', color='#26577C', alpha=0.7, label= 'No churn');
    df[churn].groupby(x_col_name)[y_col_name].mean().plot(kind='bar', color='#E55604', alpha=0.7, label= 'churn');
    
    plt.title(title, fontsize = 20)
    plt.xlabel(xlabel,fontsize = 15)
    plt.ylabel(ylabel, fontsize = 15)
    plt.legend();
    plt.grid(visible=True);

In [ ]:
# making a function to plot a bar chart of the proportion of each group against 1 feature
def plot_proportions(df, col_name, width, title= None, xlabel= None, ylabel=None):
    counts = df.groupby(['Churn', col_name]).count().sort_index().iloc[:,0]
    total_counts = df.groupby('Churn').count().sort_index().iloc[:,0]
    churn_proportions = counts[1] / total_counts[1]
    no_churn_proportions = counts[0] / total_counts[0]
    category_devider = np.arange(len(no_churn_proportions))
    

    # plot bars
    no_churn_bars = plt.bar(category_devider, no_churn_proportions, width, alpha=1, label= 'No churn', color= '#26577C')
    churn_bars = plt.bar(category_devider + width, churn_proportions, width, alpha=1, label= 'Churn', color= '#E55604')

    # title and labels
    plt.title(title, fontsize = 20)
    plt.xlabel(xlabel,fontsize = 15)
    plt.ylabel(ylabel, fontsize = 15)
    plt.grid(visible= True);
    locations = category_devider + width / 2  # xtick locations
    labels = np.sort(df[col_name].unique())  # xtick labels
    plt.xticks(locations, labels)

    # legend
    plt.legend();

------------------------

In [ ]:
plt.scatter(df['MonthlyCharges'], df['TotalCharges'], c= df['Churn'], cmap= 'coolwarm',edgecolors='k')
plt.title('The relaionship between MonthlyCharges and TotalCharges', fontsize = 20)
plt.xlabel('MonthlyCharges',fontsize = 15)
plt.ylabel('TotalCharges', fontsize = 15)
plt.grid(visible=True);
plt.show()

**Notes:**
- Most churned customers have high monthly charges but low total charges, this means that they churn early in thier tenure.

In [ ]:
#grouping customers' tenures into 4 quantiles 
df['tenure_bins'] = pd.qcut(df['tenure'].sort_values(), q=4, labels=['Q1', 'Q2', 'Q3', 'Q4'])

In [ ]:
plt.figure(figsize=(18, 6))

plt.subplot(1, 2, 1)
bar_2d(df, 'tenure_bins', 'MonthlyCharges', 'Relationship between tenure bins and mean MonthlyCharges', 'Tenure bins', 'Mean MonthlyCharges')
plt.subplot(1, 2, 2)
bar_2d(df, 'tenure_bins', 'TotalCharges', 'Relationship between tenure bins and mean TotalCharges', 'Tenure bins', 'Mean TotalCharges')

plt.tight_layout()
plt.show()

**Notes:**
- The average monthly and total charges in churned customer are higher than not-churned customers across all tenure bins,  
**Which means:** as the  customer charges gets higher, the propablity of churn gets higher too

In [ ]:
plt.figure(figsize=(18, 10))

plt.subplot(2, 2, 1)
bar_2d(df, 'OnlineSecurity', 'MonthlyCharges','Relationship between OnlineSecurity and mean MonthlyCharges', 'OnlineSecurity', 'Mean MonthlyCharges')
plt.subplot(2, 2, 2)
bar_2d(df, 'OnlineBackup', 'MonthlyCharges','Relationship between OnlineBackup and mean MonthlyCharges', 'OnlineBackup', 'Mean MonthlyCharges')
plt.subplot(2, 2, 3)
bar_2d(df, 'DeviceProtection', 'MonthlyCharges','Relationship between DeviceProtection and mean MonthlyCharges', 'DeviceProtection', 'Mean MonthlyCharges')
plt.subplot(2, 2, 4)
bar_2d(df, 'TechSupport', 'MonthlyCharges','Relationship between TechSupport and mean MonthlyCharges', 'TechSupport', 'Mean MonthlyCharges')


plt.tight_layout()
plt.show()

**Notes:**
- Churn customers that don't subscribe to services on average pay significantly higher than non-churn customers.

In [ ]:
plt.figure(figsize=(18, 10))

plt.subplot(2, 2, 1)
bar_2d(df, 'gender', 'MonthlyCharges','Relationship between gender and mean MonthlyCharges', 'Gender', 'Mean MonthlyCharges')
plt.subplot(2, 2, 2)
bar_2d(df, 'SeniorCitizen', 'MonthlyCharges','Relationship between SeniorCitizen and mean MonthlyCharges', 'SeniorCitizen', 'Mean MonthlyCharges')
plt.subplot(2, 2, 3)
bar_2d(df, 'Partner', 'MonthlyCharges','Relationship between having a Partner and mean MonthlyCharges', 'Partner', 'Mean MonthlyCharges')
plt.subplot(2, 2, 4)
bar_2d(df, 'Dependents', 'MonthlyCharges','Relationship between having Dependents and mean MonthlyCharges', 'Dependents', 'Mean MonthlyCharges')


plt.tight_layout()
plt.show()

**Notes:**
- Mean Monthly Charges are highr for churn customers along all demogarphic metrics,  
**Which indicates** an association between monthly charges and churn

----------------------

### **Key Insights**
<a id='Key-Insights'></a>


-------------

1. **Customer Distribution**:
   - Gender and partner status have no significant effect on churn.
   - Most customers are not senior citizens, and many have dependents.

2. **Service Subscription Patterns**:
   - A small percentage of customers lack phone service.
   - Most customers do not subscribe to optional services like Online Security, Online Backup, Device Protection, Tech Support, Streaming TV, or Streaming Movies.

3. **Churn Characteristics**:
   - **Higher Churn Rates**:
     - Senior citizens, single people, and those without dependents are more likely to churn.
     - Customers using fiber optic internet show the highest churn rates.
     - Customers with month-to-month contracts and paperless billing have significantly higher churn rates.
     - Most churned customers have higher monthly charges but lower total charges, indicating early churn.

   - **Lower Churn Rates**:
     - Yearly or biyearly contract customers are less likely to churn.
     - Customers with online security and related services (e.g., Online Backup, Device Protection, Tech Support) have lower churn rates.
   - **No Significant Impact**:
     - Phone service, multiple lines, and streaming services (TV or movies) do not show significant influence on churn.

4. **Payment and Billing**:
   - Most churned customers use electronic checks, while non-churned customers are evenly distributed across payment methods.
   - Customers with paperless billing are more prone to churn.

5. **Tenure and Charges**:
   - Churn rates decrease as customer tenure increases.
   - Churned customers tend to have higher monthly charges across all demographic groups.
   - Churn likelihood increases with higher monthly and total charges.


### **Recommenditions**
<a id='Recommenditions'></a>
------------

1. **Enhance Customer Retention for Senior Citizens and Singles**:
   - Develop targeted retention programs for senior citizens and single customers, offering incentives like discounts or personalized plans to reduce churn rates in these segments.

2. **Promote Long-Term Contracts**:
   - Encourage customers to switch from month-to-month contracts to longer-term contracts (yearly or biyearly) by offering attractive discounts or bundled services, as longer contracts show lower churn rates.

3. **Increase Value for High-Charge Customers**:
   - Provide more value for customers with higher monthly charges, such as premium services or loyalty rewards. These customers are at a higher risk of early churn, so adding perceived value could improve retention.

4. **Improve Service Uptake**:
   - Highlight the benefits of subscribing to services like Online Security, Tech Support, and Device Protection, as customers who use these services show lower churn rates.

5. **Address Payment Method Issues**:
   - Investigate why electronic check users are more prone to churn and consider offering alternative payment options with better incentives for these customers.

6. **Targeted Campaigns for High-Tenure Customers**:
   - Create targeted campaigns for new customers to reach high-tenure status, as higher tenure correlates with lower churn rates. Focus on providing early value and engagement to retain them longer.

7. **Analyze and Address Billing Preferences**:
   - Assess why paperless billing is associated with higher churn and explore ways to improve the experience for customers using paperless billing, such as more user-friendly payment interfaces or billing transparency.

By addressing these key areas, the company can reduce churn, improve customer satisfaction, and increase long-term revenue.


-----------------------------
## 6- Data Preprocessing and Feature Engineering
<a id='Data-Preprocessing-and-Feature-Engineering'></a>

### One-Hot Encoding and Creating Dummy Variables:

In [ ]:
df = df.drop('tenure_bins', axis=1)

In [ ]:
# One-hot encode the multi-category columns
df = pd.get_dummies(df, columns=['InternetService', 'Contract', 'PaymentMethod'], drop_first=True)
# 'drop_first=True' ensures that one category is dropped to avoid multicollinearity
# Convert all boolean columns to integers (True -> 1, False -> 0)
df = df.astype({col: 'int64' for col in df.select_dtypes(include='bool').columns})

In [ ]:
print(df.info())
print(df.shape)

In [ ]:
df.describe()

In [ ]:
X= df.drop(['Churn'],axis=1)
y = df['Churn']
X.info()

In [ ]:
df_n = df.drop('customerID', axis=1)
plt.figure(figsize=(14,7))
df_n.corr()['Churn'].sort_values(ascending = False)

### Splitting Data:

In [ ]:
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.1, random_state=101)
Test_CustomerIDs = X_test['customerID']
X_train.drop(['customerID'],axis=1,inplace=True)
X_test.drop(['customerID'],axis=1,inplace=True)

In [ ]:
def distplot(feature, frame, color=main_color[1]):
    plt.figure(figsize=(8,3))
    plt.title("Distribution for {}".format(feature))
    ax = sns.distplot(frame[feature], color= color)

In [ ]:
num_cols = ["tenure", 'MonthlyCharges', 'TotalCharges']
for feat in num_cols: distplot(feat, df)
plt.show()

**Notes:**
- Since the numerical features are distributed over different value ranges, I will use standard scalar to scale them down to the same range.

### Scaling Data:

In [ ]:
df_std = pd.DataFrame(StandardScaler().fit_transform(df[num_cols].astype('float64')),
                       columns=num_cols)
for feat in num_cols: distplot(feat, df_std, color=main_color[0])
plt.show()

In [ ]:
scaler = StandardScaler()
scaled_X_train = scaler.fit_transform(X_train)
scaled_X_test = scaler.transform(X_test)

--------------
# 7- Machine Learning Modeling
<a id="Machine-Learning-Modeling"></a>

## KNN
<a id="KNN"></a>

In [ ]:
test_error_rates = []


for k in range(1,30):
    knn_model = KNeighborsClassifier(n_neighbors=k)
    knn_model.fit(scaled_X_train,y_train)

    y_pred_test = knn_model.predict(scaled_X_test)

    test_error = 1 - accuracy_score(y_test,y_pred_test)
    test_error_rates.append(test_error)

In [ ]:
plt.plot(range(1,30),test_error_rates)
plt.show() 

#### Model Evaluation

In [ ]:
Final_model = KNeighborsClassifier(n_neighbors=17)
Final_model.fit(scaled_X_train,y_train)

In [ ]:
y_pred = Final_model.predict(scaled_X_test)
y_pred_proba = Final_model.predict_proba(scaled_X_test)

In [ ]:
confusion_matrix(y_test,y_pred)

In [ ]:
print(classification_report(y_test,y_pred))

#### Predict the non churned Customers who possibly churn:

In [ ]:
# Select non-churned customers and remove unnecessary columns
# Ensuring no unexpected columns are passed
nonchurned_customers = df[df['Churn'] == 0].drop(['customerID', 'Churn', 'Churn_Probability'], axis=1, errors='ignore')

# Scale the features
nonchurned_customers_scaled = scaler.transform(nonchurned_customers)

# Predict churn probability for non-churned customers
nonchurned_churn_probabilities = Final_model.predict_proba(nonchurned_customers_scaled)[:, 1]

# Assign churn probabilities to the original DataFrame using .loc
df.loc[df['Churn'] == 0, 'Churn_Probability'] = nonchurned_churn_probabilities

# Display selected columns for non-churned customers
df.loc[df['Churn'] == 0, ['customerID', 'Churn', 'Churn_Probability']].head()

#### Using Grid search to choose the best K :

In [ ]:
from sklearn.pipeline import Pipeline
from sklearn.model_selection import GridSearchCV
operations = [('scaler',StandardScaler()),('knn',KNeighborsClassifier())]
pipe = Pipeline(operations)
k_values = list(range(1,30))
param_grid = {'knn__n_neighbors': k_values}
full_cv_classifier = GridSearchCV(pipe,param_grid,cv=5,scoring='recall')
full_cv_classifier.fit(X_train,y_train)
full_cv_classifier.best_estimator_.get_params()

### Classification Confusion Metrix

In [ ]:
cm = confusion_matrix(y_test, y_pred)

# Create the heatmap
heatmap = go.Figure(data=go.Heatmap(
    z=cm,
    x=['Not Churn', 'Churn'],  
    y=['Churn', 'Not Churn'],        
    colorscale='Cividis',
    text=cm,
    texttemplate="%{text}",
    hoverinfo='text'
))

heatmap.update_layout(
    title='RANDOM FOREST CONFUSION MATRIX',
    xaxis_title='Predicted Label',
    yaxis_title='True Label',
    font=dict(size=14),
    width=700, 
    height=600
)

## Random Forest
<a id="Random-Forest"></a>

#### Feature Selection

- Separates the target variable (y) from the features. It drops the `customerID` and `Churn` columns from the features DataFrame (df2).

In [ ]:
# Separate the target variable from the features
y = df['Churn']  # Target
df2 = df.drop(columns=['customerID', 'Churn'])  # Drop non-predictive feature and target

# Convert categorical columns to numerical using one-hot encoding
df2 = pd.get_dummies(df2, drop_first=True)

# Define features
X = df2  # Features

- Applies one-hot encoding to convert categorical columns in df2 into numerical format, removing the first category to avoid multicollinearity.

In [ ]:
# Convert categorical columns to numerical using one-hot encoding
df2 = pd.get_dummies(df2, drop_first=True)

- Calculates and Visualizes the correlation matrix of the features to check for multicollinearity using a heatmap.

In [ ]:
# Step 2: Check for multicollinearity
cmp = 'Spectral'
plt.figure(figsize=(15, 15))
correlation_matrix = X.corr()  # Calculate correlation on the features only
sns.heatmap(correlation_matrix,cbar=True, annot=True, fmt=".2f", square=True, annot_kws={'size': 11}, cmap=cmp)
plt.title('Correlation Matrix')
plt.show()

- Splits the data into training and testing sets, using 30% of the data for testing and 70% for training.

In [ ]:
# Step 3: Split data into training and testing sets
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.3, random_state=42)

-  Initializes and Fits a `Random Forest classifier` to the training data to obtain feature importances.

In [ ]:
# Step 4: Fit the initial Random Forest model to get feature importances
model = RandomForestClassifier(random_state=42)
model.fit(X_train, y_train)

- Extracts and Displays the feature importances from the trained Random Forest model, sorting them in descending order.

In [ ]:
# Get feature importance
feature_importances = model.feature_importances_
feature_names = X.columns
importance_df = pd.DataFrame({'Feature': feature_names, 'Importance': feature_importances})
importance_df = importance_df.sort_values(by='Importance', ascending=False)

print("Feature Importances:\n", importance_df)

- Selects the features with importance greater than 0.05 and creates a new DataFrame (X_selected) containing only these important features.

In [ ]:
# Step 5: Select important features (e.g., importance > 0.05)
selected_features = importance_df[importance_df['Importance'] > 0.05]['Feature']
X_selected = X[selected_features]

print("Selected Features:\n", X_selected)

#### Random Forest Model Implementation

- Selects relevant features from the DataFrame df, Removes duplicate entries based on customerID, Converts the Churn column to a categorical type,  
  and prints the class distribution of churn.

In [ ]:
# Selected features
selected_features = ["customerID", "TotalCharges", "MonthlyCharges", "tenure", "Churn"]

# Filter the data
data_imputed = df[selected_features].drop_duplicates(subset='customerID')
data_imputed['Churn'] = data_imputed['Churn'].astype('category')

# Check the class distribution
print("Class distribution in the original dataset:\n", data_imputed['Churn'].value_counts())

- Performs a stratified split of the dataset into training and testing sets while maintaining the class distribution of churn.  
   It also removes the `customerID` column from both sets for modeling purposes.

In [ ]:
# Stratified split to maintain class distribution
sss = StratifiedShuffleSplit(n_splits=1, test_size=0.2, random_state=123)
for train_index, test_index in sss.split(data_imputed, data_imputed['Churn']):
    train_set = data_imputed.iloc[train_index]
    test_set = data_imputed.iloc[test_index]

# Drop customerID from training and testing sets
train_set = train_set.drop(columns=['customerID'])
test_CIF = test_set['customerID']
test_set = test_set.drop(columns=['customerID'])

- Checks if both classes ('Yes' and 'No') are present in the training set. If one class is missing, it raises an error and prints the class distribution.

In [ ]:
# Check if both classes are present after the split
train_set_majority = train_set[train_set['Churn'] == 0]
train_set_minority = train_set[train_set['Churn'] == 1]

# Add a check to ensure both classes are present
if len(train_set_majority) == 0 or len(train_set_minority) == 0:
    print("Class distribution in training set:\n", train_set['Churn'].value_counts())
    raise ValueError("One of the classes is missing in the training set.")

- Upsamples the minority class (customers who churned) so that it matches the size of the majority class (customers who did not churn).  
  The two sets are then concatenated into a new training set.

In [ ]:
# Upsample the minority class
train_set_minority_upsampled = resample(train_set_minority,
                                        replace=True,
                                        n_samples=len(train_set_majority),
                                        random_state=123)

train_set_upsampled = pd.concat([train_set_majority, train_set_minority_upsampled])

- Prepares the features (X_train) and target variable (y_train) for training.  
  It then initializes and fits a Random Forest classifier on the upsampled training set.

In [ ]:
# Train Random Forest model
X_train = train_set_upsampled.drop(columns=['Churn'])
y_train = train_set_upsampled['Churn']

rf_model = RandomForestClassifier(n_estimators=100, max_features=3, random_state=123)
rf_model.fit(X_train, y_train)

- Makes predictions on the test set using the trained model.     
It calculates predicted probabilities and classes, then generates and prints a confusion matrix to evaluate model performance.


In [ ]:
# Predictions on the test set
X_test = test_set.drop(columns=['Churn'])
y_test = test_set['Churn']

predicted_probs = rf_model.predict_proba(X_test)
predicted_classes = rf_model.predict(X_test)

# Confusion matrix
confusion = confusion_matrix(y_test, predicted_classes)
print("Confusion Matrix:\n", confusion)

- Calculates various performance metrics (accuracy, precision, recall, F1-score) based on the predictions made on the test set and prints them out.

In [ ]:
# Metrics calculations
accuracy = accuracy_score(y_test, predicted_classes)
precision = precision_score(y_test, predicted_classes, pos_label=1)
recall = recall_score(y_test, predicted_classes, pos_label=1)
f1 = f1_score(y_test, predicted_classes, pos_label=1)

print(f"Accuracy: {accuracy:.4f}")
print(f"Precision: {precision:.4f}")
print(f"Recall: {recall:.4f}")
print(f"F1-Score: {f1:.4f}")

- Predicts churn probabilities for customers who did not churn. It formats these probabilities as percentages and creates a DataFrame with customer IDs and predictions.

In [ ]:
# Predictions for customers who didn't churn
nochurn = data_imputed[data_imputed['Churn'] == 0].drop(columns=['Churn', 'customerID'])
nochurn_pred = rf_model.predict_proba(nochurn)[:, 1]
threshold = 0.5
binary_outcomes = (nochurn_pred >= threshold).astype(int)

# Format probabilities
formatted_probs = [f"{prob * 100:.2f}%" for prob in nochurn_pred]

predicted_data = pd.DataFrame({
    'customerID': data_imputed[data_imputed['Churn'] == 0]['customerID'],
    'churn_pred': binary_outcomes,
    'predicted_outcome': formatted_probs
})

- Merges the predictions with original customer data based on customerID, removes duplicates, displays the resulting DataFrame, and saves it to a CSV file.

In [ ]:
# Merge and filter duplicates
predicted_churn = pd.merge(data_imputed, predicted_data, on='customerID').drop_duplicates()

# Display the merged predicted data
print(predicted_churn)

# Save the predicted_churn DataFrame to a CSV file
predicted_churn.to_csv('predicted_churn.csv', index=False)

- This cell converts formatted probability strings back into numeric values.   
  It then counts how many customers are predicted to churn with a probability greater than 50% and prints this count.  
  This structured approach will help you understand each part of your code better while also making it easier to debug or modify specific sections as needed.

In [ ]:
# Convert the predicted_outcome strings to numeric values
predicted_churn['predicted_probability'] = predicted_churn['predicted_outcome'].str.replace('%', '').astype(float) / 100

# Count how many will churn with probability greater than 50%
churn_count = predicted_churn[predicted_churn['predicted_probability'] > 0.5].shape[0]

print(f"Number of customers predicted to churn with probability greater than 50%: {churn_count}")

-----------------
# 8- Summary
<a id="8--Summary"></a>

- **Contract & Monthly Charges**   
Customers who choose a 'Two-year' contract have a greater chance of not churn. While the 'Month-to-month' contract has the opposite effect. This is related to the Monthly Charges feature, where the higher the monthly fee, the greater the chance of customers to churn. To overcome this, companies need to rearrange their pricing strategies for their products to minimize the chances of churn for customers who choose the 'Month-to-month' contract.



- **Internet Service**  
If customers use 'fiber optic' internet services, the chances of customers to churn are greater. Based on insights from EDA that have been conducted, competitors are the biggest reason why people who use fiber optics churn, companies are required to optimize their fiber optic products so that they can compete more with competitor products.



- **Number of Dependents**   
The more 'Number of Dependents' the greater the chance of customers not to unsubscribe. Marketing can run campaigns to married customers rather than single ones, and try to improve sales strategies so that single customers are more interested in the products offered.



- **Tenure Months**   
The greater the 'Tenure Months', the greater the opportunity for customers not to churn, a large 'Tenure Months' indicates that our product has been well received by customers, and marketing needs to improve retention campaign strategies so that tenure months are greater.



- **Payment Method**
Customers who use electronic payment methods are more likely to churn. While electronic payments are generally convenient, this trend might indicate that these customers face issues with our online payment system or experience concerns about security and reliability. To address this, we should enhance the user experience of our electronic payment systems and provide clear, step-by-step guidance and support. Additionally, our marketing and customer support teams should actively engage with these customers to understand their pain points and offer solutions that build trust and ease of use.